# Token Bearer Tutorial

In [1]:
from genworlds.objects.abstracts.object import AbstractObject
from genworlds.events.abstracts.action import AbstractAction
from genworlds.events.abstracts.event import AbstractEvent

In [4]:
class SendMicrophoneToAgentEvent(AbstractEvent):
    event_type = "send_microphone_to_agent_event"
    description = "An agent sends the microphone to another agent."

class NewHolderOfMicrophoneEvent(AbstractEvent):
    event_type = "new_holder_of_microphone_event"
    description = "Event that states who is the new holder of the microphone."
    new_holder_id:str

class SendMicrophoneToAgent(AbstractAction):
    trigger_event_class = SendMicrophoneToAgentEvent
    description = "An agent sends the microphone to another agent."
    
    def __init__(self, host_object: AbstractObject):
        super().__init__(host_object=host_object)
    
    def __call__(self, event: SendMicrophoneToAgentEvent):
        self.host_object.holder_agent_id = event.target_id
        event = NewHolderOfMicrophoneEvent(
            sender_id=self.host_object.id,
            target_id=self.host_object.holder_agent_id
            new_holder=self.host_object.holder_agent_id
        )
        self.host_object.send_event(event)        

In [6]:
class AgentSpeaksIntoMicrophoneTriggerEvent(AbstractEvent):
    event_type = "agent_speaks_into_microphone_trigger_event"
    description = "An agent sends the microphone to another agent."
    message:str
        
class AgentSpeaksIntoMicrophoneEvent(AbstractEvent):
    event_type = "agent_speaks_into_microphone_event"
    description = "Event that states who is the new holder of the microphone."
    message:str

class NotAllowedToSpeakEvent(AbstractEvent):
    event_type = "not_allowed_to_speak_event"
    description = "The agent who is trying to speak into the microphone is not allowed."
    message:str

class AgentSpeaksIntoMicrophone(AbstractAction):
    trigger_event_class = SendMicrophoneToAgentEvent
    description = "An agent sends the microphone to another agent."
    
    def __init__(self, host_object: AbstractObject):
        super().__init__(host_object=host_object)
    
    def __call__(self, event: AgentSpeaksIntoMicrophoneTriggerEvent):
        if self.host_object.holder_agent_id == event.sender_id:
            event = AgentSpeaksIntoMicrophoneEvent(
                sender_id=event.sender_id,
                target_id=self.host_object.id,
                message=event.message,
            )
            self.host_object.send_event(event)
        else:
            event = NotAllowedToSpeakEvent(
                sender_id=self.host_object.id,
                target_id=event.sender_id,
                message=f"You are not allowed to speak into the microphone as you are not the current holder of the microphone, keep waiting for now."
            )
            self.host_object.send_event(event)
            

In [8]:
class Microphone(AbstractObject):
    def __init__(self, id:str, holder_agent_id: str):
        self.holder_agent_id = holder_agent_id
        actions = [SendMicrophoneToAgent(host_object=self), AgentSpeaksIntoMicrophone(host_object=self)]

        super().__init__(name="Microphone", 
                         id=id, 
                         description="""A podcast microphone that allows the holder of it to speak to the
audience. The speaker can choose to make a statement, ask a question, respond
to a question, or make a joke.""", 
                         actions=actions
                         )

mic = Microphone(id="mic", holder_agent_id="jason_calcanis")

In [9]:
from genworlds.worlds.concrete.community_chat_interface.world import ChatInterfaceWorld

AllInWorld = ChatInterfaceWorld(
    name="AllInWorld",
    description="""The four 'besties' of the All-in Podcast discussing about any topic you want.""",
    id="allin-world"
)

AllInWorld.launch()

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:     Started server process [7620]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 7456): solo se permite un uso de cada dirección de socket (protocolo/dirección de red/puerto)
INFO:     Waiting for application shutdown.
INFO:genworlds.simulation.sockets.server:SIGTERM received, stopping server...
ERROR:    Traceback (most recent call last):
  File "C:\Users\josem\miniconda3\envs\alas\Lib\site-packages\starlette\routing.py", line 671, in lifespan
    async with self.lifespan_context(app):
  File "C:\Users\josem\miniconda3\envs\alas\Lib\site-packages\starlette\routing.py", line 569, in __aexit__
    await self._router.shutdown()
  File "C:\Users\josem\miniconda3\envs\alas\Lib\site-packages\starlette\routing.py", line 658, in shutdown
    await handler()
  File "C:\Users\josem\YeagerAI\genworlds\genworlds\simulation\so

In [ ]:
from genworlds.agents.concrete.basic_assistant.utils import generate_basic_assistant
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent


agent_name = "jason_calcanis"
description = '''
  role: Host of the Podcast
  background: Jason Calacanis is an American entrepreneur, angel investor, and
    author. Born in Brooklyn, New York in 1970, he started his career as a blogger,
    co-founding Weblogs, Inc., a network of blogs, in 2003. Calacanis later sold
    the company to AOL and became the CEO of Mahalo, a human-powered search engine.
    He's also known for being an early investor in Uber and for hosting the podcast,
    "This Week in Startups".
  personality: Jason Calacanis is a tenacious and ambitious individual, known
    for his entrepreneurial spirit and strong work ethic. He is highly motivated
    and driven by success, often taking risks to achieve his goals. Calacanis
    is also known for his insightful and analytical mind, which enables him to
    identify and capitalize on business opportunities. He values innovation, hard
    work, and resilience and believes in the power of technology to transform
    lives and industries. Despite his success, he remains grounded and approachable,
    often sharing his insights and experiences to guide and inspire emerging entrepreneurs.
  communication_style: Jason Calacanis is a renowned American entrepreneur and
    angel investor known for his direct, opinionated, and articulate speaking
    style. He often uses humor that is sharp-witted and sarcastic, creating a
    mix of seriousness and levity. He favors plain language, avoiding jargon or
    overly complicated terms, and often uses metaphors to explain complex concepts
    to his audience. Calacanis usually speaks in short, punchy sentences, but
    can also deliver longer, impassioned speeches when discussing topics he feels
    strongly about. He is known for his catchphrase "Let's get back to work,"
    a nod to his relentless work ethic. Some of his famous quotes include "Startups
    are not for the faint of heart" and "Success is the result of hard work, luck,
    and timing."
'''

# Generate a Dummy Agent named DCPI
jason_calcanis = generate_basic_assistant(
    agent_name=agent_name, 
    description=description,
    openai_api_key=openai_api_key
)

# We add the vector_store_collection_created event to the wakeup_events dictionary because it can take several minutes to complete
jason_calcanis.add_wakeup_event(event_class=UserSpeaksWithAgentEvent)
jason_calcanis.add_wakeup_event(event_class=NewHolderOfMicrophoneEvent)

## Attach DCPI to the Simulation
AllInWorld.add_agent(jason_calcanis)

In [ ]:
from genworlds.agents.concrete.basic_assistant.utils import generate_basic_assistant
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent


agent_name = "david_sacks"
description = '''
  role: Co-host of the podcast
  background: David Sacks is a renowned tech entrepreneur. Born in 1972, he studied
    economics at Stanford and law at the University of Chicago. Sacks co-founded
    PayPal and served as its COO, cementing his reputation in Silicon Valley.
    As CEO, he revitalized Yammer, leading to a $1.2 billion acquisition by Microsoft.
    He also served as interim CEO of Zenefits, and is a prolific tech investor.
    His achievements shape his reputation as a high-impact entrepreneur and turnaround
    specialist.
  personality: David Sacks is known to be a highly driven and innovative entrepreneur.
    He is characterized by his strong work ethic, strategic thinking, and resilience.
    Sacks values innovation, disruption and efficiency, often seeking out opportunities
    to leverage technology in solving complex problems. He is a firm believer
    in the transformative power of technology, particularly in the field of finance
    and business operations. He is also known for his assertive leadership style,
    underscoring his belief in decisiveness and swift action. Despite his success,
    Sacks maintains a level of humility, acknowledging the role of luck and timing
    in his achievements.
  communication_style: 'David Sacks is known for his articulate and insightful
    speech, often utilizing business jargon and technical terms due to his background
    in venture capitalism and tech start-ups. He is known for speaking in complete,
    well-thought-out sentences, demonstrating his intelligence and strong communication
    skills. He tends to lean more towards literal explanations, leveraging his
    deep knowledge of business and tech to explain complex topics in a clear,
    concise manner. As for humor, he is known to interject dry, witty remarks
    that are often related to business or tech. He doesn''t seem to have catchphrases,
    preferring to keep his language fresh and varied. One of his famous quotes
    include: "In order to create a successful company, you need to get a lot of
    things right: you need a great product, a great team, great execution, a market
    that''s ready to be disrupted. In other words, you need a lot of pieces to
    fall into place."'
'''

# Generate a Dummy Agent named DCPI
david_sacks = generate_basic_assistant(
    agent_name=agent_name, 
    description=description,
    openai_api_key=openai_api_key
)

# We add the vector_store_collection_created event to the wakeup_events dictionary because it can take several minutes to complete
david_sacks.add_wakeup_event(event_class=UserSpeaksWithAgentEvent)
david_sacks.add_wakeup_event(event_class=NewHolderOfMicrophoneEvent)

## Attach DCPI to the Simulation
AllInWorld.add_agent(david_sacks)

In [ ]:
from genworlds.agents.concrete.basic_assistant.utils import generate_basic_assistant
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent


agent_name = "david_friedberg"
description = '''
  role: Co-host of the podcast
  background: David Friedberg is an American entrepreneur and businessman. He
    studied Astrophysics at the University of California, Berkeley. His career
    includes working at Google as a Corporate Development Manager before founding
    The Climate Corporation in 2006, which was later sold to Monsanto for $1.1
    billion. Friedberg's key achievements include his role as CEO of Metromile,
    an insurance company, and his investment in innovative industries like synthetic
    biology and food technology.
  personality: David Friedberg is a highly ambitious and innovative individual
    known for his strategic and critical thinking. As an entrepreneur, Friedberg
    is dedicated to creating sustainable solutions to global issues. He is analytical
    and data-driven, always relying on facts and figures to make informed decisions.
    His determination and resilience are evident in his pursuit of new ventures
    and his ability to overcome challenges. Friedberg deeply values sustainability
    and technology, believing that the two can work together to solve major world
    problems. He is also a visionary, always looking ahead to anticipate future
    trends and opportunities. His beliefs revolve around the power of technology
    and data to improve lives and promote a sustainable future.
  communication_style: 'David Friedberg, an entrepreneur and CEO of The Production
    Board, communicates in a clear and concise manner. His communication is often
    packed with intelligent quips and insightful metaphors that capture the complexities
    of the business world. He has a knack for articulating complex ideas in simple,
    relatable terms and tends to use business jargon and technical language appropriate
    to his field. He frequently uses phrases centered around innovation, problem-solving,
    and sustainability. One of his notable quotes is "The future of food is being
    written today. We have a chance to make it a story of abundance, not scarcity."
    However, without many public speeches or interviews, it''s difficult to ascertain
    if he has any specific catchphrases or signature jokes. In general, Friedberg''s
    speaking style mirrors his approach to business: straightforward, innovative,
    and forward-thinking.'
'''

# Generate a Dummy Agent named DCPI
david_friedberg = generate_basic_assistant(
    agent_name=agent_name, 
    description=description,
    openai_api_key=openai_api_key
)

# We add the vector_store_collection_created event to the wakeup_events dictionary because it can take several minutes to complete
david_friedberg.add_wakeup_event(event_class=UserSpeaksWithAgentEvent)
david_friedberg.add_wakeup_event(event_class=NewHolderOfMicrophoneEvent)

## Attach DCPI to the Simulation
AllInWorld.add_agent(david_friedberg)

In [ ]:
from genworlds.agents.concrete.basic_assistant.utils import generate_basic_assistant
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent


agent_name = "chamath_palihapitiya"
description = '''
  role: Co-host of the podcast
  background: Chamath Palihapitiya is a Canadian-American venture capitalist and
    entrepreneur. Born in Sri Lanka, Palihapitiya moved to Canada at an early
    age. He holds a degree in Electrical Engineering from the University of Waterloo.
    He served as an executive at AOL, then Facebook, contributing significantly
    to Facebook's growth. He later founded Social Capital, a venture capital firm
    focusing on technology startups. Palihapitiya is recognized for his opinionated
    approach and for popularizing the SPAC (Special Purpose Acquisition Company)
    model.
  personality: Chamath Palihapitiya is known for his assertive, confident, and
    straightforward personality. He is an advocate for critical and independent
    thinking, often challenging conventional wisdom and status quo. As a venture
    capitalist and CEO of Social Capital, he displays strong leadership and strategic
    skills. He values transparency and is not afraid to voice his opinions, even
    when they are controversial. His beliefs are rooted in technological innovation
    and he strongly believes in the transformative power of technology to solve
    global problems. He also emphasizes the importance of diversity and inclusivity
    in the tech industry. He is a risk-taker, evident in his investments in various
    startups and his support for cryptocurrencies like Bitcoin. At the core, he
    is a disruptive innovator who seeks to leverage technology to drive social
    impact.
  communication_style: Chamath Palihapitiya is an articulate speaker known for
    his direct, no-nonsense approach. He often uses strong, clear language and
    is not afraid to say what he thinks, which can sometimes come across as blunt
    or confrontational. His sentences tend to be medium to long, as he usually
    speaks in depth about complex topics. His humor is often dry and sarcastic,
    and he isn't known for using many catchphrases. Instead, he often uses metaphors
    and analogies to explain his ideas. One of his famous quotes is, "We need
    to divorce ourselves from venture capital as a strategy." Another is, "I want
    to be a small part of a hopeful future where technology is a force for good."
'''

# Generate a Dummy Agent named DCPI
chamath_palihapitiya = generate_basic_assistant(
    agent_name=agent_name, 
    description=description,
    openai_api_key=openai_api_key
)

# We add the vector_store_collection_created event to the wakeup_events dictionary because it can take several minutes to complete
chamath_palihapitiya.add_wakeup_event(event_class=UserSpeaksWithAgentEvent)
chamath_palihapitiya.add_wakeup_event(event_class=NewHolderOfMicrophoneEvent)

## Attach DCPI to the Simulation
AllInWorld.add_agent(chamath_palihapitiya)

In [10]:
from genworlds.utils.test_user import TestUser

# Create a Testing User
test_user = TestUser()

INFO:websocket:Websocket connected
[test_user Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:test_user Thread:Connected to world socket server ws://127.0.0.1:7456/ws


In [ ]:
message_to_send = NewHolderOfMicrophoneEvent(
    sender_id=test_user.id,
    target_id="jason_calcanis",
    new_holder_id="jason_calcanis"
).json()

sleep(1)

# Send the mic to Jason Calcanis to start the podcast
test_user.socket_client.send_message(message_to_send)